In [17]:
import ast
import os
from operator import itemgetter
from apps.patterns.analyzer import BaseFileAnalyzer
from apps.structure import get_full_path

class ServiceDocAnalyzer(BaseFileAnalyzer):
    def __init__(self, *args, **kwargs):
        self.doc_elements = []
        super(ServiceDocAnalyzer, self).__init__(*args, **kwargs)
        
    def build_doc(self, payload):
        self.doc_elements.append(payload)
    
    def get_analysis(self):
        elements = sorted(self.doc_elements, key=itemgetter('name'))
        avg_score = sum(list(map(lambda x: x["score"], elements))) / len(elements)
        return {
            "services": elements,
            "global": {
                "score": avg_score
            }
        }
    
    def get_function_score(self, payload):
        score = 100
        if not payload.get('has_docstring'):
            score -= 33
            
        arguments = payload.get('arguments', [])
        if len(arguments) > 0:
            missing_args = 0
            for arg in arguments:
                if arg.get("type", "missing") == "missing":
                    missing_args += 1
            score_to_remove = 33*(missing_args/len(arguments)) 
            score -= score_to_remove
        
        if payload.get("return_type") == "missing":
            score -= 33
        return score
    
    def get_processed_type(self, annotation):
        if annotation is None:
            return "missing"
        if annotation.__class__ == ast.Name:
            return annotation.id
        if annotation.__class__ == ast.Attribute:
            return annotation.attr
        if annotation.__class__ == ast.Subscript:
            inner_type = self.get_processed_type(annotation.slice)
            outer_type = self.get_processed_type(annotation.value)
            return f"{outer_type}[{inner_type}]"
        if annotation.__class__ == ast.Str:
            return annotation.s
        if annotation.__class__ == ast.Num:
            raise Exception("oops", annotation, dir(annotation))
        if annotation.__class__ == ast.Constant:
            raise Exception("oops", annotation, dir(annotation))
        if annotation.__class__ == ast.NameConstant:
            return annotation.value
        if annotation.__class__ == ast.Index:
            index_type = self.get_processed_type(annotation.value)
            return index_type
        if annotation.__class__ == ast.Tuple:
            types = []
            for expr in annotation.elts:
                elts_type = self.get_processed_type(expr)
                types.append(elts_type)
            return ", ".join(types)
        raise Exception("oops", annotation, dir(annotation))
        
    def visit_FunctionDef(self, node):
        arguments = [
            {
                "name": arg.arg,
                "type": self.get_processed_type(arg.annotation),
            }
            for arg in node.args.args
        ]
        docstring = ast.get_docstring(node)
        return_type = self.get_processed_type(node.returns)
        payload = dict(
            name=node.name,
            has_docstring=False if docstring is None or docstring == "" else True,
            docstring=docstring,
            arguments=arguments,
            return_type=return_type
        )
        score = self.get_function_score(payload)
        payload["score"] = score
        self.build_doc(payload)
        

file_path = "apps/catalog/services.py"
full_file_path = get_full_path(file_path=file_path)

anlyzr = ServiceDocAnalyzer(entrypoint=full_file_path)
anlyzr.start()
anlyzr.get_analysis()

{'services': [{'name': '__init__',
   'has_docstring': False,
   'docstring': None,
   'arguments': [{'name': 'self', 'type': 'missing'},
    {'name': 'user', 'type': 'missing'},
    {'name': 'lat', 'type': 'missing'},
    {'name': 'lng', 'type': 'missing'},
    {'name': 'locality', 'type': 'missing'},
    {'name': 'country_code', 'type': 'missing'},
    {'name': 'language', 'type': 'missing'}],
   'return_type': 'missing',
   'score': 1.0},
  {'name': '__init__',
   'has_docstring': False,
   'docstring': None,
   'arguments': [{'name': 'self', 'type': 'missing'},
    {'name': 'product_branch', 'type': 'missing'}],
   'return_type': 'missing',
   'score': 1.0},
  {'name': '__init__',
   'has_docstring': False,
   'docstring': None,
   'arguments': [{'name': 'self', 'type': 'missing'}],
   'return_type': 'missing',
   'score': 1.0},
  {'name': '__init__',
   'has_docstring': False,
   'docstring': None,
   'arguments': [{'name': 'self', 'type': 'missing'}],
   'return_type': 'missing',